In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from itertools import cycle
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy import interp
import matplotlib.pyplot as plt
from engarde.decorators import has_dtypes
from capstone_project import preprocessor as pre

#from IPython.display import display
%matplotlib inline

In [2]:
# Make sure that the loaded dataframe has the correct layout otherwise throw assertion error 
@has_dtypes(dict(question1=object, question2=object, is_duplicate=int, q1_tokens=object, q2_tokens=object))
def load_data(file_dir, filename):
    """Load dataframe using filename as input. A pandas dataframe is returned and it is checked that it 
    has the correct layout.
    """
    df = pd.read_pickle(file_dir+filename)
    return df

In [3]:
# The current working directory for python is the capstone_project/notebook folder
file_directory = "../output/data/"
prefix = ""

train_data = load_data(file_directory, prefix+"train_data.pkl")
y = train_data["is_duplicate"].values

In [4]:
model_directory = "../output/models/"
skf = pre.load_pickle(model_directory, "kfolds.pkl") 

In [5]:
#tfidf = pre.TfidfTransformer()
import gensim
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("../data/GoogleNews-vectors-negative300.bin.gz", binary=True)

word_features = pre.FeatureTransformer()
#word2vec_transform = pre.Word2vecTransformer(word2vec_model, sum_up=True)
word2vec_transform = pre.Word2vecTransformer()
word2vec_features = pre.VectorFeatureTransformer()
mms = MinMaxScaler()
clf = LogisticRegression()

In [6]:


#feature_creator = FeatureUnion([('feature_engineering', feature_engineering), ('tfidf', tfidf)])
#pipe = Pipeline([("features", feature_creator), ("logistic", clf)])

word2vec_pipe = Pipeline([("vec_transformer", word2vec_transform), ("vec_features", word2vec_features)])
feature_creator = FeatureUnion([('word_features', word_features), ('word2vec_pipeline', word2vec_pipe)])
pipe = Pipeline([("features", feature_creator), ("scaling", mms), ("logistic", clf)])

print pipe.get_params().keys()

['features__word2vec_pipeline__vec_features', 'features', 'features__word2vec_pipeline__vec_transformer', 'scaling__feature_range', 'logistic__C', 'features__word_features', 'features__n_jobs', 'logistic__max_iter', 'logistic__tol', 'logistic__dual', 'logistic__fit_intercept', 'scaling__copy', 'features__transformer_weights', 'logistic__multi_class', 'features__transformer_list', 'logistic__penalty', 'logistic__solver', 'features__word2vec_pipeline__vec_transformer__sum_up', 'scaling', 'features__word2vec_pipeline__steps', 'logistic__random_state', 'logistic__intercept_scaling', 'logistic__verbose', 'logistic__n_jobs', 'features__word2vec_pipeline__vec_transformer__model', 'logistic__warm_start', 'steps', 'logistic__class_weight', 'logistic', 'features__word2vec_pipeline']


In [7]:
#word2vec_pipe.fit(train_data, y)
#test = feature_creator.fit_transform(train_data, y) 
#fit_params = {"features__vectransform__model": word2vec_model}
#pipe.fit(train_data, y, **fit_params)

In [8]:
#http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
from scipy.stats import uniform
from time import time


#print cross_val_score(pipe, X, y, scoring="neg_log_loss", cv=skf)
#print cross_val_score(pipe, X, y, scoring="accuracy", cv=skf)

param_dist = {"logistic__C": uniform(3, 8),}
n_iter_search = 20
random_search = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=n_iter_search, cv=skf, scoring="neg_log_loss")

print random_search.get_params().keys()

initial_params = {"estimator__features__word2vec_pipeline__vec_transformer__sum_up": True, 
                  "estimator__features__word2vec_pipeline__vec_transformer__model": word2vec_model}
random_search.set_params(**initial_params) 
    
start = time()
random_search.fit(train_data, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)  

#for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #y_train, y_test = y[train_index], y[test_index]

['n_jobs', 'verbose', 'estimator__logistic__n_jobs', 'estimator__features__n_jobs', 'estimator__features__word2vec_pipeline__vec_features', 'estimator__steps', 'estimator__logistic__intercept_scaling', 'estimator__scaling__feature_range', 'estimator__features__word2vec_pipeline__vec_transformer__model', 'cv', 'scoring', 'estimator__features__word2vec_pipeline__vec_transformer', 'estimator__logistic__fit_intercept', 'estimator__features__word2vec_pipeline__steps', 'random_state', 'estimator__logistic__solver', 'pre_dispatch', 'estimator__logistic__tol', 'estimator__scaling__copy', 'estimator__logistic__max_iter', 'estimator__features__word2vec_pipeline', 'estimator__logistic__multi_class', 'param_distributions', 'estimator__features__transformer_list', 'estimator__features__transformer_weights', 'n_iter', 'fit_params', 'estimator__logistic__verbose', 'refit', 'iid', 'estimator__logistic__warm_start', 'estimator__logistic__C', 'estimator__features__word2vec_pipeline__vec_transformer__sum

/home/andre/software/anaconda3/envs/capstone_project/lib/python2.7/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))
/home/andre/software/anaconda3/envs/capstone_project/lib/python2.7/site-packages/scipy/spatial/distance.py:616: RuntimeWarning: invalid value encountered in double_scalars
  np.double(np.bitwise_or(u != 0, v != 0).sum()))
/home/andre/software/anaconda3/envs/capstone_project/lib/python2.7/site-packages/scipy/spatial/distance.py:810: RuntimeWarning: invalid value encountered in double_scalars
  return abs(u - v).sum() / abs(u + v).sum()


In [ ]:
prefix = "second_test_"

save_pickle(random_search, model_directory, prefix+"logistic_search.pkl") 
save_pickle(pipe, model_directory, prefix+"logistic_pipeline.pkl") 

In [ ]:
prefix = "first_test_"

random_search = load_pickle(model_directory, prefix+"logistic_search.pkl")
pipe = load_pickle(model_directory, prefix+"logistic_pipeline.pkl")

In [ ]:
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html



best_pipe = random_search.best_estimator_

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)

colors = cycle(['cyan', 'indigo', 'seagreen'])
lw = 2

accs = []
i = 0
for (train_index, test_index), color in zip(skf.split(train_data, y), colors):
    pred = best_pipe.predict_proba(train_data.iloc[test_index])
    fpr, tpr, thresholds = roc_curve(y[test_index], pred[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=color,
             label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    
    acc = best_pipe.score(train_data.iloc[test_index], y[test_index])
    accs.append(acc)
    print "ROC fold {} accuracy: {:.4f}".format(i, acc)
    i += 1
    
print "Mean accuracy: {:.4f} (std: {:.4f})".format(np.mean(accs), np.std(accs))
plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',
         label='Luck')

mean_tpr /= skf.get_n_splits(train_data, y)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='g', linestyle='--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()